In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [2]:
# On identifie le processus par le processus de Poisson composé, qui finit à un temps fixé T
# On crée des fonctions pour modéliser le processus.

#### On définit nu, la loi des incréments
# Ancien : plus lent
saut_1_ancien = lambda x: np.random.choice([-1, 1], size=x, replace=True, p=[0.5, 0.5]) #correspond à m=1
saut_2_ancien = lambda x: np.random.choice([-3, -2, -1, 1, 2, 3], size=x, replace=True, p=0.5*np.array([1/6, 1/3, 1/2, 1/2, 1/3, 1/6])) #correspond à m=3

# Nouveau : beaucoup plus vite
value_1 = np.array([-1, 1])
value_2 = np.array([-3, -2, -2, -1, -1, -1, 1, 1, 1, 2, 2, 3])
saut_1 = lambda x : value_1[np.random.randint(low=2, size=x)]
saut_2 = lambda x : value_2[np.random.randint(low=12, size=x)]

##  Q2




### Q2-1 Probabilité de ruine

In [3]:
#Monte-Carlo accéléré - Méthode 2
from numba import jit
from numpy.random import exponential, randint, poisson
from numpy import array
from tqdm.notebook import tqdm

@jit(nopython=True)
def Q2_1_MC(P0, T, m):
  if m == 1:
    value = array([-1, 1])
    low = 2
  else:
    value = array([-3, -2, -2, -1, -1, -1, 1, 1, 1, 2, 2, 3])
    low = 12
  J2_0 = array([-1, 1])[randint(low=2)]

  somme = P0
  T1 = exponential(660)
  T2 = exponential(110)
  while T1 < T:
    while T2 < T1:
      somme += J2_0
      if somme < 0:
        return 1
      J2_0 *= -1
      T2 += exponential(110)
    somme += value[randint(low=low)]
    if somme < 0:
      return 1
    T1 += exponential(660)
  return 0

def Q2_1_MC_n(P0, T, m, size):
  somme = 0
  for i in tqdm(range(size)):
    somme += Q2_1_MC(P0, T, m)
  return somme / size

In [4]:
%%time
P0=35
T = 4*3600
m = 3
M=int(1e7)

p_r = Q2_1_MC_n(P0, T, m, M)

R_IC = 1.96*np.sqrt(p_r*(1-p_r))/np.sqrt(M) #rayon de l'intervalle de confiance
print(p_r, "+/-", R_IC)
#print("Estimation de la probabilité de ruine pour m=3 par méthode de Monte-Carlo accélérée: {:09.8f}+/-{:09.8f}".format(p_r,R_IC))

#P0=35, M=10^5, m=3: p_r = 3 10^-5 et R_IC = 3.4 10^-5 => faut augmenter M
#Estimation de la probabilité de ruine pour m=3 par méthode de Monte-Carlo accélérée: 0.00005200+/-0.00001413


5.05e-05 +/- 4.404440974284932e-06
CPU times: user 1min 1s, sys: 159 ms, total: 1min 2s
Wall time: 1min 1s


m = 1 : Le résultat est très petit, on a utilisé un fichier Q2_1-Monte-Carlo.py pour le faire tourner sur un terminal séparé.

Le résultat est ... pour M = int(1e10)

In [11]:
from scipy.stats import norm
quantiles = [1e-4, 1 - 1e-4, 1e-5, 1 - 1e-5, 1e-6, 1 - 1e-6]
P0 = 35
lamb = 1 / 660
T = 4 * 60 * 60
ms = [1, 3]
for m in ms:
  for quantile in quantiles:
    if m == 1:
      sigma_carre = 1
    elif m == 3:
      sigma_carre = 10 / 3
    print("quantile {} for m = {} : {}".format(quantile, m, norm.ppf(quantile, loc=35, scale=np.sqrt(lamb * T * sigma_carre))))

quantile 0.0001 for m = 1 : 17.62849755652073
quantile 0.9999 for m = 1 : 52.3715024434794
quantile 1e-05 for m = 1 : 15.078722119801292
quantile 0.99999 for m = 1 : 54.921277880203455
quantile 1e-06 for m = 1 : 12.796785166577937
quantile 0.999999 for m = 1 : 57.20321483339492
quantile 0.0001 for m = 3 : 3.2841208465009686
quantile 0.9999 for m = 3 : 66.71587915349927
quantile 1e-05 for m = 3 : -1.3711108977117732
quantile 0.99999 for m = 3 : 71.37111089772044
quantile 1e-06 for m = 3 : -5.53733871132858
quantile 0.999999 for m = 3 : 75.53733871127902


#### Echantillonnage d'importance

In [4]:
def ruine(lamb1,lamb2,T,P0,c,increment1_c,esp):
    #returns 0,0 if no price is strictly negative
    #returns 1, 1/L_T otherwise

    t1=np.random.exponential(1/lamb1/esp)
    t2=np.random.exponential(1/lamb2)
    P = P0
    Pmin=P0

    sumIncrements1=0
    increment2 = -1+2*np.random.binomial(1,0.5) #pour savoir si le premier incrément du processus 2 vaut 1 ou -1

    while t1<T or t2<T:
        if P<Pmin:
            Pmin=P

        if (t2<t1):
            P+=increment2
            increment2 *=-1
            t2+=np.random.exponential(1/lamb2)
        else:
            increment1= increment1_c(1)
            sumIncrements1+=increment1
            P+=increment1
            t1+=np.random.exponential(1/lamb1/esp)

    if Pmin<0:
        return 1,np.exp(lamb1*T*(esp-1)-c*sumIncrements1)
    return 0,0

def echant_imp(M,lamb1,lamb2,T,P0,c,m): 
    liste_ruine = np.zeros(M) 
    liste_poids = np.zeros(M) #liste des 1/L_T pour l'échantillonnage d'importance
    
    if m==1:
        esp = np.cosh(c) #E[exp(c J_1)]
        increment1_c = lambda x: np.random.choice([-1,1],p=np.array([np.exp(-c),np.exp(c)])/(2*esp))

    if m==3:
        esp = np.cosh(c)/2 +np.cosh(2*c)/3 + np.cosh(3*c)/6
        increment1_c = lambda x: np.random.choice([-3,-2,-1,1,2,3],p=np.array([np.exp(-3*c)/12,np.exp(-2*c)/6,np.exp(-c)/4,np.exp(c)/4,np.exp(2*c)/6,np.exp(3*c)/12])/esp)

    for l in range(M):
        liste_ruine[l],liste_poids[l] = ruine(lamb1,lamb2,T,P0,c,increment1_c,esp) 
    proba_prix_negatif = np.mean(liste_ruine*liste_poids)
    R_IC = 1.96*np.sqrt(np.mean(liste_ruine*liste_poids*liste_poids) - proba_prix_negatif**2)/np.sqrt(M)
    return np.mean(liste_ruine),proba_prix_negatif,R_IC #il faut que np.mean(liste_ruine) soit environ 0.5 pour que c soit bien réglé


In [8]:
%%time
M=int(1e7)
c=-1.3
m=1
P0=35
T=4*60*60
lamb1 = 1/660
lamb2 = 1/110

fraction_negatifs,proba_prix_negatif,R_IC = echant_imp(M,lamb1,lamb2,T,P0,c,m)
print("La probabilité de prix négatif pour m={} est {} +/- {}".format(m,proba_prix_negatif,R_IC))
print("On a choisi c = {}, ce qui donne comme proportion de prix négatifs : {} (cette proportion devrait être proche de 0.5)".format(c,fraction_negatifs))

#La probabilité de prix négatif pour m=1 est 1.5381890748570916e-12 +/- 6.387042482281324e-14
#On a choisi c = -1.3, ce qui donne comme proportion de prix négatifs : 0.60214 (cette proportion devrait être proche de 0.5)

La probabilité de prix négatif pour m=1 est 1.4887029896232638e-12 +/- 6.513949115417591e-15
On a choisi c = -1.3, ce qui donne comme proportion de prix négatifs : 0.6007454 (cette proportion devrait être proche de 0.5)
CPU times: user 4h 13min 50s, sys: 10min 6s, total: 4h 23min 56s
Wall time: 4h 13min 30s


In [9]:
c = -0.4
m=3
fraction_negatifs,proba_prix_negatif,R_IC = echant_imp(M,lamb1,lamb2,T,P0,c,m)
print("La probabilité de prix négatif pour m={} est {} +/- {}".format(m,proba_prix_negatif,R_IC))
print("On a choisi c = {}, ce qui donne comme proportion de prix négatifs : {} (cette proportion devrait être proche de 0.5)".format(c,fraction_negatifs))

La probabilité de prix négatif pour m=3 est 5.0975247656192116e-05 +/- 1.1977057028361074e-07
On a choisi c = -0.4, ce qui donne comme proportion de prix négatifs : 0.4515575 (cette proportion devrait être proche de 0.5)
